In [1]:
import open3d as o3d
import numpy as np
import os
import k3d
import glob

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Set up paths
folder_name = "brown_bm_1"

color_folder = f"../datasets/SUN3d/{folder_name}/image"
depth_folder = f"../datasets/SUN3d/{folder_name}/depth"
extrinsics_file = f"../datasets/SUN3d/{folder_name}/extrinsics/20130512130736.txt"
output_path = f"../datasets/SUN3d/{folder_name}/combined_point_cloud.ply"
intrinsics_file = f'../datasets/SUN3d/{folder_name}/intrinsics.txt'

def load_intrinsics(filepath):
    # Read the 3x3 intrinsic matrix from file
    with open(filepath, 'r') as file:
        lines = file.readlines()
    # Parse lines into a 3x3 NumPy array
    intrinsic_matrix = np.array([list(map(float, line.split())) for line in lines])
    return intrinsic_matrix

# Path to your intrinsics file
intrinsic_matrix = load_intrinsics(intrinsics_file)
width, height = 640, 480  # Replace with actual image dimensions if different
camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(
    width=width,
    height=height,
    fx=intrinsic_matrix[0, 0],
    fy=intrinsic_matrix[1, 1],
    cx=intrinsic_matrix[0, 2],
    cy=intrinsic_matrix[1, 2]
)

In [4]:
# Load extrinsic matrices from the text file
def load_extrinsics(filepath):
    extrinsics = []
    with open(filepath, 'r') as file:
        lines = file.readlines()
    for i in range(0, len(lines), 4):  # Each extrinsic is represented by 4 lines
        matrix = np.array([list(map(float, line.split())) for line in lines[i:i+4]])
        extrinsics.append(matrix)
    return extrinsics

extrinsic_matrices = load_extrinsics(extrinsics_file)

In [5]:
# Initialize combined point cloud
combined_pcd = o3d.geometry.PointCloud()

# Load RGB and depth images
color_images = sorted(glob.glob(os.path.join(color_folder, '*.jpg')))
depth_images = sorted(glob.glob(os.path.join(depth_folder, '*.png')))
num_files = 2

for i, (color_path, depth_path, extrinsic) in enumerate(zip(color_images, depth_images, extrinsic_matrices)):
    if i >= num_files:
        break

    print(i, color_path, extrinsic)

    color_raw = o3d.io.read_image(color_path)
    depth_raw = o3d.io.read_image(depth_path)

    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_raw, depth_raw, convert_rgb_to_intensity=False
    )

    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera_intrinsic)

    pcd.transform(extrinsic)
    combined_pcd += pcd  # Add to combined point cloud

: 

In [ ]:
o3d.io.write_point_cloud(output_path, combined_pcd)

points = np.asarray(combined_pcd.points) / 10
colors = np.asarray(combined_pcd.colors) if combined_pcd.has_colors() else np.ones(points.shape)

colors = (colors * 255).astype(np.uint8)
colors = (colors[:, 0] << 16 | colors[:, 1] << 8 | colors[:, 2]).astype(np.uint32)

plot = k3d.plot()
point_cloud = k3d.points(positions=points.astype(np.float32),
                         colors=colors,
                         point_size=0.001)
plot += point_cloud
plot.display()

Output()